In [2]:
import sys
sys.path.append('../../')
import os

import pandas as pd
from utils import fill_empty, VColumns
from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load input file

In [11]:
df1 = pd.read_csv("v1-start.csv")
df1.head()

,type,external_id,name,format,verkaufspreis,vintage,kellerei,einkaufspreis,lagerbereich,bestand
0,WHITE,1.0,Cuvee Trias,"0,75 lt.","€ 49,00",2020.0,Niedrist,17.30,4.0,12.0
1,WHITE,2.0,Terlaner Sauvignon Porphyr&Kalk,"0,75 lt.","€ 49,00",2021.0,Sauvignon Niedrist,14.30,4.0,12.0
2,WHITE,5.0,Pinot Grigio Vigna Kastel Ringberg,"0,75 lt.","€ 49,00",2022.0,Elena Walch,15.65,6.0,30.0
3,WHITE,6.0,Chardonnay Magdalena,"0,75 lt.","€ 49,00",2021.0,Nicolussi Leck,10.20,4.0,30.0
4,WHITE,7.0,Pinot Bianco Platt & Riegl,"0,75 lt.","€ 35,00",2022.0,KG. Girlan,9.70,6.0,24.0


# Start elaborating columns

Create a new dataframe with empty values, so that we can start filling in values from the input dataframe, without overriding columns.

In [4]:
df1.head()

,type,external_id,name,format,verkaufspreis,vintage,kellerei,einkaufspreis,lagerbereich,bestand
0,WHITE,1.0,Cuvee Trias,"0,75 lt.","€ 49,00",2020,Niedrist,17.30,4.0,12.0
1,WHITE,2.0,Terlaner Sauvignon Porphyr&Kalk,"0,75 lt.","€ 49,00",2021,Sauvignon Niedrist,14.30,4.0,12.0
2,WHITE,5.0,Pinot Grigio Vigna Kastel Ringberg,"0,75 lt.","€ 49,00",2022,Elena Walch,15.65,6.0,30.0
3,WHITE,6.0,Chardonnay Magdalena,"0,75 lt.","€ 49,00",2021,Nicolussi Leck,10.20,4.0,30.0
4,WHITE,7.0,Pinot Bianco Platt & Riegl,"0,75 lt.","€ 35,00",2022,KG. Girlan,9.70,6.0,24.0


In [13]:
# create new empty dataframe
df = pd.DataFrame(columns=VColumns.v2())
df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible


In [14]:
# copy values from original columns to new columns
df['external_id'] = df1['external_id']
df['name'] = df1['name']
df['winery_name'] = df1['kellerei']
# df["size"] = df1["size"] handle size after
df["vintage"] = df1["vintage"].fillna(0)
df["price"] = df1["verkaufspreis"].apply(lambda x: int(float(str(x).replace('€','').replace(',','.').strip())*100) if not pd.isnull(x) else 0)
df["purchase_price"] = df1["einkaufspreis"].apply(lambda x: int(float(str(x).replace('€','').replace(',','.').strip())*100) if not pd.isnull(x) else 0)
df["quantity"] = df1["bestand"]
df["storage_area"] = df1["lagerbereich"]
df["visible"] = True

df.tail()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible
491,905.0,NaN,Riesling Auslese Graacher Domprobst,"Willy Schäfer, Mosel",NaN,NaN,2006.0,17000,3500,3.0,1.0,NaN,True
492,1214.0,NaN,Don Zolio Pedro Ximenez Very Sweet Sherry Dolce,Williams&Humbert,NaN,NaN,0.0,4500,1500,6.0,3.0,NaN,True
493,1215.0,NaN,Dulce Monastrel,Olivares Jumilla,NaN,NaN,1996.0,6500,1000,3.0,1.0,NaN,True
494,NaN,NaN,Steinbock Sparkling ZERO Hofstätter,Hofstätter,NaN,NaN,0.0,3500,730,1.0,1.0,NaN,True
495,NaN,NaN,Steinbock Riesling ZERO Hofstätter,Hofstätter,NaN,NaN,0.0,3500,730,6.0,1.0,NaN,True


In [19]:
# list sizes
sizes = df1["format"].unique()
sizes

# map sizes
size_map = {
    "0,375 lt.": "HALF_BOTTLE",
    "0,500 lt.": "HALF_LITER",
    "0,50 lt.": "HALF_LITER",
    "0,75 lt.": "BOTTLE",
    "1,50 lt.": "MAGNUM",
    "1,5 lt.": "MAGNUM",
    "3,00 lt.": "JEROBOAM",
    "3,0 lt.": "JEROBOAM",
}


array(['0,75 lt.', '0,375 lt.', '0,500 lt.', '1,50 lt.', '1,5 lt.',
       '3,00 lt.', '3,0 lt.', '0,50 lt.', '0,5 lt.', '0,750 lt.'],
      dtype=object)

In [22]:
# size
df['size'] = df1["size"].fillna("BOTTLE")

# convert prices to cents
df['price'] = 0
df['purchase_price'] = df1['purchase_price'].apply(lambda x: int(x*100) if not pd.isnull(x) else 0) # convert to cents

df.tail()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible
215,NaN,NaN,Haderburg Hausmanof Riserva,,NaN,BOTTLE,NaN,0,0,NaN,Cantinetta,,True
216,NaN,NaN,Rocche dei Manzoni Valentino Brut Elena,,NaN,BOTTLE,NaN,0,0,NaN,Cantinetta,,True
217,NaN,NaN,Rocche dei Manzoni Valentino Brut 0,,NaN,BOTTLE,NaN,0,0,NaN,Cantinetta,,True
218,NaN,NaN,Rocche dei Manzoni Valentino Brut 0 Rosè,,NaN,BOTTLE,NaN,0,0,NaN,Cantinetta,,True
219,NaN,NaN,Francesco Bellei Metodo Classico,,NaN,BOTTLE,NaN,0,0,NaN,Cantinetta,,True


In [23]:
df = fill_empty(df, VColumns.v2(), False)
df.tail()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible
215,,,Haderburg Hausmanof Riserva,,,BOTTLE,0,0,0,0,Cantinetta,,True
216,,,Rocche dei Manzoni Valentino Brut Elena,,,BOTTLE,0,0,0,0,Cantinetta,,True
217,,,Rocche dei Manzoni Valentino Brut 0,,,BOTTLE,0,0,0,0,Cantinetta,,True
218,,,Rocche dei Manzoni Valentino Brut 0 Rosè,,,BOTTLE,0,0,0,0,Cantinetta,,True
219,,,Francesco Bellei Metodo Classico,,,BOTTLE,0,0,0,0,Cantinetta,,True


# Merge files and write output

If there are multiple files, merge them into one and create single output file.

In [24]:
df.head()

df_out = pd.concat([df], ignore_index=True)
df_out.to_csv("v2-cleaned.csv", index=False)

bool()

False